In [1021]:
from collections import defaultdict
import pandas as pd
from scipy.stats import pearsonr
import numpy as np
import math
from sklearn.metrics import pairwise_distances

#STORE SIMILARITIES IN A DICTIONARY SIM (THIS IS LIKE A SPARSE-MATRIX WHERE WE ONLY STORE NON-ZERO VALUES)
SIM = defaultdict(dict)
#STORE RATINGS in ITM
ITM = defaultdict(dict)

#df=pd.read_csv("ratings-train.txt", sep=",", header =0, index_col=['movie_id', 'user_id'], usecols=['movie_id','user_id', 'ratings'])
df=pd.read_csv("ratings-train.txt", sep=",", header =0, usecols=['movie_id','user_id', 'ratings'])
pqr=df
#print(pqr.head)
#df.sort_values('user_id', ascending=True, inplace=False)

#print(df)
#df.values
missing_pivot=df.pivot_table(values='ratings', index='user_id', columns='movie_id').fillna(0).T
missing_pivot

#'''.T since we need to peform the item-item collaborative filtering'''
#'''missing_pivot includes all the training data with missing values =0
#it has 35 movies and 2572 unique users.'''
#print(missing_pivot)

user_id,0,1,2,3,4,5,6,7,8,9,...,2563,2564,2565,2566,2567,2568,2569,2570,2571,2572
movie_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1023]:
'''step 1: calculaitng similarities'''
# corr_mat = np.corrcoef(missing_pivot.T)
# print(corr_mat)
#using corr_mat works but I'm using pairwise_distances to make the output more user-friendly. 
#print(len(corr_mat))
#print(pearsonr(missing_pivot, missing_pivot))
from sklearn.metrics import pairwise_distances


cosine_sim = 1-pairwise_distances(missing_pivot, metric="correlation")
ab=pd.DataFrame(cosine_sim)
ab
#pd.DataFrame(cosine_sim)

#'''ab contains a 35 X 35 table consisting of the similarities of one movie to another.'''

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1.000000,0.068655,0.090340,0.112779,0.032681,0.044680,0.048188,0.113437,0.030228,0.106017,...,0.078151,0.072292,0.102090,0.075762,0.095737,0.048832,0.054514,-0.000623,0.001256,0.046626
1,0.068655,1.000000,0.008190,0.010431,-0.004632,-0.000931,0.000753,0.029399,-0.009071,0.028491,...,0.001012,-0.024663,-0.002957,0.022511,0.028138,-0.012354,0.004070,-0.040001,0.007264,-0.002468
2,0.090340,0.008190,1.000000,0.054014,0.039860,0.008363,0.119035,0.046084,0.001201,0.007217,...,0.059958,0.003220,0.041945,-0.003060,0.018896,-0.000427,0.009143,-0.034518,-0.006911,0.008592
3,0.112779,0.010431,0.054014,1.000000,0.007948,0.014135,0.001871,0.128710,0.008731,0.033358,...,0.027893,0.030129,0.055755,0.003275,0.030176,-0.001730,0.037199,-0.022863,-0.022310,0.075294
4,0.032681,-0.004632,0.039860,0.007948,1.000000,0.032678,0.012773,0.007725,-0.029436,-0.004996,...,0.064439,0.029674,0.024024,-0.015068,-0.018184,-0.001227,-0.009116,-0.048455,-0.012575,-0.028599
5,0.044680,-0.000931,0.008363,0.014135,0.032678,1.000000,-0.000451,0.034952,0.035281,0.009617,...,0.013868,0.057337,-0.004645,-0.005082,0.036482,0.010941,0.038441,-0.024567,-0.013357,0.007292
6,0.048188,0.000753,0.119035,0.001871,0.012773,-0.000451,1.000000,0.007449,-0.002763,-0.009245,...,0.002028,-0.034389,-0.008201,-0.015213,0.037808,-0.009715,-0.001253,-0.061396,-0.007143,-0.031865
7,0.113437,0.029399,0.046084,0.128710,0.007725,0.034952,0.007449,1.000000,0.011278,0.026124,...,0.017414,0.028457,0.028884,-0.003956,0.038694,-0.003726,0.016017,-0.024998,-0.029246,0.048073
8,0.030228,-0.009071,0.001201,0.008731,-0.029436,0.035281,-0.002763,0.011278,1.000000,-0.010924,...,0.007977,-0.031770,-0.016770,-0.014581,0.008408,-0.034367,0.029416,-0.054578,-0.035287,-0.032752
9,0.106017,0.028491,0.007217,0.033358,-0.004996,0.009617,-0.009245,0.026124,-0.010924,1.000000,...,-0.010795,-0.006752,0.012543,0.004519,0.015356,-0.023520,0.005379,-0.041496,-0.023793,0.034358


In [1024]:
list1=[]
#to extract all movie columns
for i in pqr.movie_id:
    if i not in list1:
        list1.append(i) 
list2=list1
movie_id=list1
#ab.set_index(movie_id)

bc=dict(ab)
ab==bc

bc[1][2]
#print((list2))
SIM = defaultdict(dict)
for m in list1:
    for m1 in list2:
        if m!=m1:
            SIM[m][m1] = ab[m][m1]
weights=(SIM)
#print(SIM)
'''calculates the similarity of movie 20 to the movie 15'''
'''SIM includes a list consisting of tll the similarity ratings'''
print(SIM[20][15]) 
#print(SIM[20][15])



-0.02192228273932706


In [1025]:
ifile = open("ratings-train-Copy1.txt")
for l in ifile:
    parts = l.strip().split(",")
    ITM[int(parts[0])][int(parts[1])] = float(parts[2])
ifile.close()
list1=[]
list2=[]
dict3=dict()
for m in ITM.keys():
    for m1 in ITM.keys():
        #print([m,m1], sep=",")
        if m==m1:
            movies=list1.append(m)
            
            ITM1=dict(ITM[m])
            abc=ITM1.values()
            x=0
            for i in abc:
                x+=i
            y=len(ITM[m])
            mean = (x/y)
            '''calculating the mean ratings for every movie based on the users who've already rated it'''
            mean = list2.append(mean) 
            '''array comprising all the mean ratings for the movie'''


            mean_ratings=np.array(mean)  

for x in range(len(list1)):
    dict3[x]=list2[x]
ifile.close()
#print(((ITM.values())))
print(len(dict3))


df=pd.read_csv("ratings-test.txt", sep=",", header =0, usecols=['movie_id','user_id', 'ratings'])
abc=df.pivot_table(values='ratings', index='user_id', columns='movie_id').fillna(0).T
'''abc includes the testing data in a tabular form'''
abc

35


user_id,18,19,20,35,63,67,76,85,86,87,...,2497,2499,2502,2510,2518,2534,2538,2547,2553,2557
movie_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1026]:
# neigh = NearestNeighbors(n_neighbors=2)
# neigh.fit(SIM)
model_knn=NearestNeighbors(n_neighbors=25, algorithm='brute', metric='correlation')
model_knn.fit(missing_pivot)
#model_knn.fit(ab)

similarities, indices = model_knn.kneighbors(missing_pivot.values, n_neighbors=25)
'''predicting ratings for 25th movie by the 1473th user'''
sum1=0
product=0

#print(indices[17][2])==26, 2 is the index for the movies similar to 17. 
#(indices[17]) = [17,  5, 26, 24, 15] similar movies
#print(similarities[17][25])==0.9392647307855434, 2 is the index for the movies similar to 17.
#similarities[17] = ([0. , 0.87334923, 0.93926473, 0.94580496, 0.94835143]) similarities 
#print(missing_pivot[0][27]) = 1.0
#missing_pivot(user_id, movie_id) = rating
for i in range(len(indices[25])):
    rating=(missing_pivot[1473][i]) #1473 is the user id and i is the movie id
    similarity=similarities[25][i] #25 is the movie id and i is the index
    product+=(rating*similarity)
    if rating*similarity!=0:
        sum1+=similarity 
print(product)
print(sum1)
print(product/sum1)
#print (missing_pivot[i])
#(indices[25])
#missing_pivot[17]



18.498747900252493
18.498747900252493
1.0


In [1027]:
'''making the above algorithm into a function so that it can be referenced to later'''
def predicted_scores(x, y): #x=movie_id, y=user_id
    sum1=0
    product=0
    for i in range(len(indices[x])):
        rating=(missing_pivot[y][i]) #1473 is the user id and i is the movie id
        similarity=similarities[x][i] #25 is the movie id and i is the index
        product+=(rating*similarity)
        if rating*similarity!=0:
            sum1+=similarity 
    return product/sum1

'''25th movie, 630th user'''


predicted_scores(25, 630) 


3.4997055497999394

In [1028]:
# for i in range(len(list1)):
#     for x in range(len(list2)):
#         print(list3[i-1][x-1])
list4=[]
for i in range(len(list3)):
    
    list4.append(list3[i])
#print((list4))
print(list4[1][1]) #list4 includes complete list of [movie_id, user_id] for the testing dataset.  
print(len(list4))
#print(list3[200])
(list4)==list3

395
390


True

In [1029]:
abc=[]
print(list4[1][1])
for i in range(len(list4)):
    list4[i][0]=int(list4[i][0])
    list4[i][1]=int(list4[i][1])
    try:
        abc.append(predicted_scores(list4[i][0], list4[i][1]))
    except KeyError:
        #abc.append(0)
        abc.append(dict3[list4[i][0]]) #dict3 stores the average ratings for the movie i. if there's no rating for the user,
                             #instead of printing 0, the mean ratings would be printed so that it wouldnt affect 
                             #the mean squared error significantly.
predicted_rating=abc
 #predicted ratings has the predictions based on the nearest neighbors abd the formula.
predicted_rating = [3 if math.isnan(x) else x for x in predicted_rating]

print(len(predicted_rating))

395
390


/var/folders/rj/qvp3193x6570fbg8sc0vz99c0000gp/T/ipykernel_97243/1817629780.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return product/sum1


In [1030]:
ifile = open("ratings-test-Copy1.txt")
actual_rating=[]
i=0
for l in ifile:
    parts = l.strip().split(",")
    movie = parts[0]
    user = parts[1]
    truerating = float(parts[2])
    actual_rating.append(truerating)
print((len(actual_rating)))
#     for x in range(len(truerating)):
#         actual_rating.append(truerating)
# print(len(actual_rating))
    

390


In [1031]:
#390==390 so we can start calculating the mean squared error. 
#print(actual_rating)
#print(predicted_rating)
#print(actual_rating)
#print(predicted_rating)
for i in range(len(actual_rating)):
    actual_rating[i]=float(actual_rating[i])
#print(type(predicted_rating[1]), type(actual_rating[1]))
print('25 nearest neighbors')
from sklearn.metrics import mean_squared_error
mean_squared_error(actual_rating, predicted_rating)


25 nearest neighbors


1.838431223615865

In [1032]:
model_knn=NearestNeighbors(n_neighbors=5, algorithm='brute', metric='correlation')
model_knn.fit(missing_pivot)
similarities, indices = model_knn.kneighbors(missing_pivot.values, n_neighbors=5)


abc=[]
#print(list4[1][1])
for i in range(len(list4)):
    list4[i][0]=int(list4[i][0])
    list4[i][1]=int(list4[i][1])
    try:
        #abc.append(0)
        abc.append(predicted_scores(list4[i][0], list4[i][1]))
    except KeyError:
        abc.append(dict3[list4[i][0]]) #dict3 stores the average ratings for the movie i. if there's no rating for the user,
                             #instead of printing 0, the mean ratings would be printed so that it wouldnt affect 
                             #the mean squared error significantly.
predicted_rating=abc
 #predicted ratings has the predictions based on the nearest neighbors abd the formula.
predicted_rating = [3 if math.isnan(x) else x for x in predicted_rating]

#print(len(predicted_rating))

ifile = open("ratings-test-Copy1.txt")
actual_rating=[]
i=0
for l in ifile:
    parts = l.strip().split(",")
    movie = parts[0]
    user = parts[1]
    truerating = float(parts[2])
    actual_rating.append(truerating)
#print((len(actual_rating)))
for i in range(len(actual_rating)):
    actual_rating[i]=float(actual_rating[i])
#print(type(predicted_rating[1]), type(actual_rating[1]))
print('5 nearest neighbors')
from sklearn.metrics import mean_squared_error
mean_squared_error(actual_rating, predicted_rating)


5 nearest neighbors


/var/folders/rj/qvp3193x6570fbg8sc0vz99c0000gp/T/ipykernel_97243/1817629780.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return product/sum1


1.8506389281409217

In [1033]:
model_knn=NearestNeighbors(n_neighbors=15, algorithm='brute', metric='correlation')
model_knn.fit(missing_pivot)
similarities, indices = model_knn.kneighbors(missing_pivot.values, n_neighbors=15)

abc=[]
#print(list4[1][1])
for i in range(len(list4)):
    list4[i][0]=int(list4[i][0])
    list4[i][1]=int(list4[i][1])
    try:
        abc.append(predicted_scores(list4[i][0], list4[i][1]))
    except KeyError:
        #abc.append(0)
        abc.append(dict3[list4[i][0]]) #dict3 stores the average ratings for the movie i. if there's no rating for the user,
                             #instead of printing 0, the mean ratings would be printed so that it wouldnt affect 
                             #the mean squared error significantly.
predicted_rating=abc
 #predicted ratings has the predictions based on the nearest neighbors abd the formula.
predicted_rating = [3 if math.isnan(x) else x for x in predicted_rating]

#print(len(predicted_rating))

ifile = open("ratings-test-Copy1.txt")
actual_rating=[]
i=0
for l in ifile:
    parts = l.strip().split(",")
    movie = parts[0]
    user = parts[1]
    truerating = float(parts[2])
    actual_rating.append(truerating)
#print((len(actual_rating)))
for i in range(len(actual_rating)):
    actual_rating[i]=float(actual_rating[i])
#print(type(predicted_rating[1]), type(actual_rating[1]))
print('15 nearest neightbors')
from sklearn.metrics import mean_squared_error
mean_squared_error(actual_rating, predicted_rating)

/var/folders/rj/qvp3193x6570fbg8sc0vz99c0000gp/T/ipykernel_97243/1817629780.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return product/sum1


15 nearest neightbors


1.8515508712180442

In [1034]:
#Note that the way I have computed my predictions, if there's no value predicted within the neighboorhood, 
#instead of giving it a null or 0, i gave it the average rating for the movie based on all the users who've
#rated it. as a result, there isn't a significant mean squared error difference between the neighborhoods. However, 
#if there wasn't any implementation like that, the errors would have been much more greater. 
#given below i'll show the mean squared error if all the missing values from the neighborhood would be 0 instead of the avg
#for neighborhood sizes 5, 15 and 25 respectively and demonstrate the high errors. 

In [1035]:
model_knn=NearestNeighbors(n_neighbors=5, algorithm='brute', metric='correlation')
model_knn.fit(missing_pivot)
similarities, indices = model_knn.kneighbors(missing_pivot.values, n_neighbors=5)

abc=[]
#print(list4[1][1])
for i in range(len(list4)):
    list4[i][0]=int(list4[i][0])
    list4[i][1]=int(list4[i][1])
    try:
        abc.append(predicted_scores(list4[i][0], list4[i][1]))
    except KeyError:
        abc.append(0)
        #abc.append(dict3[list4[i][0]]) #dict3 stores the average ratings for the movie i. if there's no rating for the user,
                             #instead of printing 0, the mean ratings would be printed so that it wouldnt affect 
                             #the mean squared error significantly.
predicted_rating=abc
 #predicted ratings has the predictions based on the nearest neighbors abd the formula.
predicted_rating = [3 if math.isnan(x) else x for x in predicted_rating]

#print(len(predicted_rating))

ifile = open("ratings-test-Copy1.txt")
actual_rating=[]
i=0
for l in ifile:
    parts = l.strip().split(",")
    movie = parts[0]
    user = parts[1]
    truerating = float(parts[2])
    actual_rating.append(truerating)
#print((len(actual_rating)))
for i in range(len(actual_rating)):
    actual_rating[i]=float(actual_rating[i])
#print(type(predicted_rating[1]), type(actual_rating[1]))
print('5 nearest neightbors with missing values = 0 instead of the avg gives a pretty high error')
from sklearn.metrics import mean_squared_error
mean_squared_error(actual_rating, predicted_rating)

5 nearest neightbors with missing values = 0 instead of the avg gives a pretty high error


/var/folders/rj/qvp3193x6570fbg8sc0vz99c0000gp/T/ipykernel_97243/1817629780.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return product/sum1


7.419433403840417

In [1036]:
model_knn=NearestNeighbors(n_neighbors=15, algorithm='brute', metric='correlation')
model_knn.fit(missing_pivot)
similarities, indices = model_knn.kneighbors(missing_pivot.values, n_neighbors=15)


abc=[]
#print(list4[1][1])
for i in range(len(list4)):
    list4[i][0]=int(list4[i][0])
    list4[i][1]=int(list4[i][1])
    try:
        abc.append(predicted_scores(list4[i][0], list4[i][1]))
    except KeyError:
        abc.append(0)
        #abc.append(dict3[list4[i][0]]) #dict3 stores the average ratings for the movie i. if there's no rating for the user,
                             #instead of printing 0, the mean ratings would be printed so that it wouldnt affect 
                             #the mean squared error significantly.
predicted_rating=abc
 #predicted ratings has the predictions based on the nearest neighbors abd the formula.
predicted_rating = [3 if math.isnan(x) else x for x in predicted_rating]

#print(len(predicted_rating))

ifile = open("ratings-test-Copy1.txt")
actual_rating=[]
i=0
for l in ifile:
    parts = l.strip().split(",")
    movie = parts[0]
    user = parts[1]
    truerating = float(parts[2])
    actual_rating.append(truerating)
#print((len(actual_rating)))
for i in range(len(actual_rating)):
    actual_rating[i]=float(actual_rating[i])
#print(type(predicted_rating[1]), type(actual_rating[1]))
print('15 nearest neightbors with missing values = 0 instead of the avg')
from sklearn.metrics import mean_squared_error
mean_squared_error(actual_rating, predicted_rating)

/var/folders/rj/qvp3193x6570fbg8sc0vz99c0000gp/T/ipykernel_97243/1817629780.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return product/sum1


15 nearest neightbors with missing values = 0 instead of the avg


7.42034534691754

In [1039]:
model_knn=NearestNeighbors(n_neighbors=25, algorithm='brute', metric='correlation')
model_knn.fit(missing_pivot)
similarities, indices = model_knn.kneighbors(missing_pivot.values, n_neighbors=25)

abc=[]
#print(list4[1][1])
for i in range(len(list4)):
    list4[i][0]=int(list4[i][0])
    list4[i][1]=int(list4[i][1])
    try:
        abc.append(predicted_scores(list4[i][0], list4[i][1]))
    except KeyError:
        abc.append(0)
        #abc.append(dict3[list4[i][0]]) #dict3 stores the average ratings for the movie i. if there's no rating for the user,
                             #instead of printing 0, the mean ratings would be printed so that it wouldnt affect 
                             #the mean squared error significantly.
predicted_rating=abc
 #predicted ratings has the predictions based on the nearest neighbors abd the formula.
predicted_rating = [3 if math.isnan(x) else x for x in predicted_rating]

#print(len(predicted_rating))

ifile = open("ratings-test-Copy1.txt")
actual_rating=[]
i=0
for l in ifile:
    parts = l.strip().split(",")
    movie = parts[0]
    user = parts[1]
    truerating = float(parts[2])
    actual_rating.append(truerating)
#print((len(actual_rating)))
for i in range(len(actual_rating)):
    actual_rating[i]=float(actual_rating[i])
#print(type(predicted_rating[1]), type(actual_rating[1]))
print('25 nearest neightbors with missing values = 0 instead of the avg gives a comparatively low error')
from sklearn.metrics import mean_squared_error
mean_squared_error(actual_rating, predicted_rating)

25 nearest neightbors with missing values = 0 instead of the avg gives a comparatively low error


/var/folders/rj/qvp3193x6570fbg8sc0vz99c0000gp/T/ipykernel_97243/1817629780.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return product/sum1


7.407225699315361